# Prediktiv analyse

## 1. Importerting og datainnlasting

For å gjøre databehandlingen ryddigere har vi lagt datainnhentingen og klargjøringen i en egen Python-fil i src-mappen (src/prediksjon.py). Der har vi definert funksjonen predikasjon1 som slår sammen temperaturdata fra Gløshaugen og luftkvalitet fra Elgeseter, basert på tidspunkt.

Vi importerer funksjonen og bruker den til å lage en felles DataFrame som inneholder alle variablene som vi trenger. Vi brukler df.head() for å teste at DataFramen blir hentet riktig. 

In [ ]:
import sys, os
sys.path.append(os.path.abspath("../"))

from src.prediksjon import prediksjon1

df = prediksjon1(
    "../data/temp_gloshaugen_historisk_renset.csv",
    "../data/gyldig_historisk_luftkvalitet.csv"
)

df.head()


## 2. Scatterplot visualisering

Nå lager vi scatterplots for å se om det er noen sammenheng eller luftkvalitet og temperatur i en visualisering av dataene vi har. Dersom det dannes et tydelig mønster, kan det tyde til at det finnes en sammenheng som kan regregeres.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(15, 4))

for i, col in enumerate(["NO2", "PM10", "PM25"]):
    plt.subplot(1, 3, i + 1)
    sns.scatterplot(data=df, x=col, y="temperatur", alpha=0.5)
    plt.title(f"Temperatur vs {col}")
    plt.xlabel(col)
    plt.ylabel("Temperatur")

plt.tight_layout()
plt.show()


## 3. Tolkning av scatterplots

### Temperatur vs NO2
Formene på punktene viser at lavt nivå av NO2 henger sammen med normaltemperatur mellom 0 og 25 grader. Noen avvik (outliers) forekommer, men punktene tyder også på at det er en mulig tendens til at høy NO2 henger sammen med lavere temperaturer. 

Punktene er generelt spredte og kan tolkes i en sammenheng med at kald luft kan henge sammen med mer NO2 i lufta.

### Temperatur vs PM10
Formene på punktene viser at det generelt er lite nivå med PM10 og at de fleste punktene ligger under 60 µg/m³. Det finnes noen få avvik (outliers). 

Det ser ikke ut til at det er noen sammenheng scatterplottemn. Outliers ligger litt overalt uten ?? til 

### Temperatur vs PM2.5
Formene på punktene ligner NO2 scatterplotten. Det finnes flere lave verdier enn høye og det kan tolkes at høy PM2.5 forekommer oftere ved lave temperaturforhold.

Selv om scatterplotsene ikke viser en tydelig sammenheng, kan vi se en potensiell sammenheng i NO2 og PM2.5 scatterplotsene. Vi velger derfor å fortsette med å lage en lineær regresjonsmodell for å undersøke om det faktisk finnes en sammenheng, eller om det er for svakt til å si noe om.


## Regresjon

Her trener vi regresjonsmodellen hvor NO2, PM10 og PM2.5 vil bli brukt som forklaringsvariablene og temperatur er målvariablen. Vi bruker scikit-learn for å bygge modellen.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# X = forklaringsvariabler (luftkvalitet)
X = df[["NO2", "PM10", "PM25"]]
# y = målvariabel (temperatur)
y = df["temperatur"]

# Lag modell og tren den
model = LinearRegression()
model.fit(X, y)

# Prediksjon
y_pred = model.predict(X)

# Evaluering
r2 = r2_score(y, y_pred)
mse = mean_squared_error(y, y_pred)

print(f"R²-verdi: {r2:.3f}")
print(f"MSE: {mse:.2f}")


### Regresjonsmodellen

## Hvor bra er den?
**R2**: 

**MSE**:

### Visualisering av faktisk vs predikert temperatur

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(5,5))
plt.scatter(y, y_pred, alpha=0.5)
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'r--')  # Diagonal linje
plt.xlabel("Faktisk temperatur")
plt.ylabel("Predikert temperatur")
plt.title("Faktisk vs Predikert temperatur")
plt.grid(True)
plt.tight_layout()
plt.show()


### Hva forteller scatterplottet?

- Om det hadde vært en perfekt predikasjon ville de blå punktene vært på linje med den røde linjen.
- **Rød linje:** Den idelle linjen som representerer hva som ville vært en perfekt predikasjon. 
- **Blå punkter:** Det modellen faktisk predikerte, sammenlignet med de faktiske temperaturene.

Vi ser tydelig at punktene ikke følger den røde linje, men heller at de ligger spredt rundt på et smalt område i midten av plottet. Med dette kan vi forstå at modellen i liten grad klarte å tilpasse seg variasjonen i temperaturene på de tre forklaringsvariablene.

R2 verdien på 5,4% bekrefter dette.


## Ekstra: Lineær regresjon med kun NO2
Ettersom at NO2 hadde den mest synlige sammenhengen med temperatur av de tre ulike forklaringsvariablene, prøver vi å trene en regresjonsmodell med denne variablen alene. Hensikten med dette er å undersøke om modellen presterer bedre eller dårligere enn når vi brukte alle tre variablene.

In [ ]:
# Kun NO2 som forklaringsvariabel
X_no2 = df[["NO2"]]
y = df["temperatur"]

# Ny modell
model_no2 = LinearRegression()
model_no2.fit(X_no2, y)

# Prediksjoner
y_pred_no2 = model_no2.predict(X_no2)

# Evaluering
r2_no2 = r2_score(y, y_pred_no2)
mse_no2 = mean_squared_error(y, y_pred_no2)

print(f"R²-verdi (kun NO₂): {r2_no2:.3f}")
print(f"MSE (kun NO₂): {mse_no2:.2f}")


In [ ]:
plt.figure(figsize=(6,6))
plt.scatter(y, y_pred_no2, alpha=0.5)
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'r--')  # Perfekt prediksjon
plt.xlabel("Faktisk temperatur")
plt.ylabel("Predikert temperatur (kun NO2")
plt.title("Faktisk vs Predikert temperatur\nModell med kun NO2")
plt.grid(True)
plt.tight_layout()
plt.show()


### Hva forteller scatterplottet?

Scatterplottet er ganske lik den tidligere regresjonsmodellen. Den predikerte temperaturen er rundt 10 grader og har liten variasjon. Den kan forstås som dårligere enn den tidligere modellen og R2 prediksjonen på 2,5% bekrefter dette.

Dette bekrefter at temperatur var lite påvirket av klimagassene NO2, PM10 og PM2.5 i 2024, og da i forhold til andre mulige forhold som vær, årstid, vind og lignende variabler.

## Sammenligning av modeller og oppsummering

lll